# Identifying Double Entries
- Our database has multiple ratings for a couple of films. Here we identified and deleted them!

In [1]:
import pickle
import pandas as pd
from sklearn.decomposition import NMF
from fuzzywuzzy import process
import numpy as np
from sqlalchemy import create_engine

In [2]:
HOST = 'localhost'
PORT = '5432'
DBNAME = 'movies'

In [4]:
connection_string_mac = f'postgres://{HOST}:{PORT}/{DBNAME}'
connection = create_engine(connection_string_mac)
df = pd.read_sql_table('movie_ratings', connection)

In [5]:
df.head()

,userid,rating,title
0,1,4.0,Toy Story (1995)
1,1,4.0,Grumpier Old Men (1995)
2,1,4.0,Heat (1995)
3,1,5.0,Seven (a.k.a. Se7en) (1995)
4,1,5.0,"Usual Suspects, The (1995)"


In [ ]:
df2 = df.groupby(['userid','title']).count()

problem_users = df2[df2['rating'] == 2].reset_index(1).index.tolist()

In [297]:
for i in range(len(problem_users)):
    movie = df[df['userid'] == problem_users[i]]['title'].value_counts().index[0]
    print(problem_users[i], movie)

111 Confessions of a Dangerous Mind (2002)
28 War of the Worlds (2005)
509 Emma (1996)
68 War of the Worlds (2005)


In [299]:
df[(df['userid'] == '28') & (df['title'] == 'War of the Worlds (2005)')]

,userid,rating,title
4590,28,3.5,War of the Worlds (2005)
4747,28,3.5,War of the Worlds (2005)


In [300]:
df[(df['userid'] == '111') & (df['title'] == 'Confessions of a Dangerous Mind (2002)')]

,userid,rating,title
17449,111,4.0,Confessions of a Dangerous Mind (2002)
17819,111,4.0,Confessions of a Dangerous Mind (2002)


In [301]:
df[(df['userid'] == '509') & (df['title'] == 'Emma (1996)')]

,userid,rating,title
80405,509,3.5,Emma (1996)
80596,509,3.5,Emma (1996)


In [302]:
df[(df['userid'] == '68') & (df['title'] == 'War of the Worlds (2005)')]

,userid,rating,title
11241,68,2.0,War of the Worlds (2005)
11451,68,2.5,War of the Worlds (2005)


In [16]:
df.set_index(['userid', 'title']).unstack(1).head()

rating                         \
title  'Hellboy': The Seeds of Creation (2004) 'Round Midnight (1986)   
userid                                                                  
1                                          NaN                    NaN   
10                                         NaN                    NaN   
100                                        NaN                    NaN   
101                                        NaN                    NaN   
102                                        NaN                    NaN   

                                                      \
title  'Salem's Lot (2004) 'Til There Was You (1997)   
userid                                                 
1                      NaN                       NaN   
10                     NaN                       NaN   
100                    NaN                       NaN   
101                    NaN                       NaN   
102                    NaN                       NaN   

                                                           \
title  'Tis the Season for Love (2015) 'burbs, The (1989)   
userid                                                      
1                                  NaN                NaN   
10                                 NaN                NaN   
100                                NaN                NaN   
101                                NaN                NaN   
102                                NaN                NaN   

                                                         \
title  'night Mother (1986) (500) Days of Summer (2009)   
userid                                                    
1                       NaN                         NaN   
10                      NaN                         NaN   
100                     NaN                         NaN   
101                     NaN                         NaN   
102                     NaN                         NaN   

                                                                 \
title  *batteries not included (1987) ...All the Marbles (1981)   
userid                                                            
1                                 NaN                       NaN   
10                                NaN                       NaN   
100                               NaN                       NaN   
101                               NaN                       NaN   
102                               NaN                       NaN   

                          ...                                              \
title                     ...                    Zulu (2013) [REC] (2007)   
userid                    ...                                               
1                         ...                            NaN          NaN   
10                        ...                            NaN          NaN   
100                       ...                            NaN          NaN   
101                       ...                            NaN          NaN   
102                       ...                            NaN          NaN   

                                              \
title  [REC]² (2009) [REC]³ 3 Génesis (2012)   
userid                                         
1                NaN                     NaN   
10               NaN                     NaN   
100              NaN                     NaN   
101              NaN                     NaN   
102              NaN                     NaN   

                                                                               \
title  anohana: The Flower We Saw That Day - The Movie (2013) eXistenZ (1999)   
userid                                                                          
1                                                     NaN                 NaN   
10                                                    NaN                 NaN   
100                                                   NaN                 NaN   
101                

In [69]:
df.iloc[4740:4750]

,userid,rating,title
4740,28,3.0,"Day the Earth Stood Still, The (2008)"
4741,28,2.5,Doubt (2008)
4742,28,4.0,Gran Torino (2008)
4743,28,4.0,Seven Pounds (2008)
4744,28,3.0,"Wrestler, The (2008)"
4745,28,4.5,"Curious Case of Benjamin Button, The (2008)"
4746,28,2.5,Valkyrie (2008)
4747,28,3.5,War of the Worlds (2005)
4748,28,3.5,Defiance (2008)
4749,28,2.5,Underworld: Rise of the Lycans (2009)


In [34]:
df.pivot()

<bound method DataFrame.pivot of       userid rating                                              title
0          1    4.0                                   Toy Story (1995)
1          1    4.0                            Grumpier Old Men (1995)
2          1    4.0                                        Heat (1995)
3          1    5.0                        Seven (a.k.a. Se7en) (1995)
4          1    5.0                         Usual Suspects, The (1995)
5          1    3.0                         From Dusk Till Dawn (1996)
6          1    5.0                               Bottle Rocket (1996)
7          1    4.0                                  Braveheart (1995)
8          1    5.0                                     Rob Roy (1995)
9          1    5.0                              Canadian Bacon (1995)
10         1    5.0                                   Desperado (1995)
11         1    5.0                               Billy Madison (1995)
12         1    3.0                         

In [8]:
df.pivot(index='userid', columns='title', values='rating') 

ValueError: Index contains duplicate entries, cannot reshape